In [ ]:
# Python version: 3.8
# Asic : Antminer L3+, S19j Pro

In [ ]:
import socket
import json
from datetime import datetime, timedelta
import requests
from requests.auth import HTTPDigestAuth
from bs4 import BeautifulSoup

In [ ]:
class asicMiner:
    description = {}
    #description_header =  ['ip','when','item_type','Elapsed','hashrate_5s', 'hashrate_av', 'miner_count', 'fan_num', 'Found Blocks','Pool1','User']
    #description_text = []
    stats = {}
    summary = {}
    pools = {}
    edevs = {}
    systeminfo = {}
    connected = False
    canConnected = False
    user=''
    password=''
    
    def __init__(self,ip):
        self.ip = ip
        #print('AsicMiner Create! ' + ip)
    
    def set_credential(self,user,password):
        self.user = user
        self.password = password
        
    def executeSocketCommand(self,command_):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        try:
            sock.connect((self.ip, 4028))
        except Exception as e:
            print (self.ip, e)
            
            return False

        data_={'command': command_}   # summary stats pools
        sock.send(bytes(json.dumps(data_), 'UTF-8'))

        response = ''
        while 1:
            buf = sock.recv(4096).decode('UTF-8')
            if buf:
                response += buf
            else:
                break

        sock.shutdown(socket.SHUT_RDWR)
        sock.close()
        result = json.loads(response[:-1].replace('}{', '},{'))
        output = json.dumps(result, indent=2)
        self.connected = True
        return result    

    def generateDescription(self):
        
        ####### STATS ########
        stats_data=self.stats
        data = {'ip': self.ip}
        status = stats_data["STATUS"][0]
        data['When'] = status['When']

        stats = stats_data["STATS"][0]
        data['item_type'] = stats['Type']


        stats = stats_data["STATS"][1]

        data['Elapsed'] = stats['Elapsed']
        data['hashrate_5s'] = stats['GHS 5s']
        data['hashrate_av'] = stats['GHS av']
        data['miner_count'] = stats['miner_count']
        data['fan_num'] = stats['fan_num']

        fan_num = 0
        for i in range(8):
            index = i+1
            key = 'fan'+str(index)
            if key in stats:
                v = stats['fan'+str(index)]
                if v:
                    fan_num += 1
                    data['fan'+str(fan_num)] = v

        ####### SUMMARY ########
        summary_data = self.summary["SUMMARY"][0]
        data['Found Blocks'] = summary_data['Found Blocks']
        
        ###### POOLS #######
        pool1_data = self.pools['POOLS'][0]
        data['Pool1'] = pool1_data['Stratum URL']
        data['User'] = pool1_data['User']
        self.description = data
        
    def description_header(self):
        #header_ =[]
        header_ = ['ip','when','item_type','Elapsed','hashrate - av', 'miner - fan', 'Found Blocks','Pool1','User']
        return header_
    
    def description_text(self):
        #text_ =[]
        d = datetime.fromtimestamp(self.description['When'])
        e =timedelta(seconds = self.description['Elapsed'])
        text_= [self.description['ip'], str(d), self.description['item_type'],str(e),\
                        str(self.description['hashrate_5s']) + ' - ' + str(self.description['hashrate_av'])  , str(self.description['miner_count']) + ' - ' + str(self.description['fan_num'])  ,\
                        self.description['Found Blocks'],self.description['Pool1'],self.description['User']]
        #self.description_text = text_
        return text_
    
    def updateStatus(self):
        
        self.stats   = self.executeSocketCommand('stats')
        self.summary = self.executeSocketCommand('summary')
        self.pools   = self.executeSocketCommand('pools')
        if(self.connected == True):
            self.generateDescription()
            self.description_text()
        #print('!!!! excute success  !!!! ')
        
    # For test connection Alive or not
    def testConnect(self):
        self.summary = self.executeSocketCommand('summary')
        #if(self.connected == True):
        #   self.generateDescription()
    
    def reboot(self):
        try:
            res= requests.get('http://' + self.ip + '/cgi-bin/reboot.cgi', auth=HTTPDigestAuth(self.user, self.password))
            #print(res)
        except:        
            print("[{0}]: Could not reboot".format(ip))
        return 0 
    
    def get_system_info(self):
        try:
            with requests.get('http://'+ self.ip +'/cgi-bin/get_system_info.cgi', auth=HTTPDigestAuth(self.user, self.password)) as r:
                self.systeminfo = r.content

        except:
            print("[{0}]: Could not initialize".format(self.ip))
    
    def get_manual_freqs(self):
        try:
            with requests.get('http://'+ self.ip +'/cgi-bin/get_freq_logs.cgi', auth=HTTPDigestAuth(self.user, self.password)) as r:
                print(r.content)
                return r.content

        except:
            print("[{0}]: Could not initialize".format(self.ip))
'''

getAjaxData("/cgi-bin/miner_summary.cgi",setSummary);//summary table
getAjaxData("/cgi-bin/miner_pools.cgi",setPools);//pools table
getAjaxData("/cgi-bin/miner_stats.cgi",setAntMiner);//antminer table
        
        /cgi-bin/minerStatus.cgi  # return html

Another Example will try later:
https://github.com/romdr/cgminer-monitor/blob/master/cgminer-monitor.py
     
     '/cgi-bin/get_manual_freqs.cgi'
     '/cgi-bin/get_freq_logs.cgi'
'''



In [ ]:
def test():
    miner = asicMiner('192.168.xx.xxx')
    miner.updateStatus()
    '''
    print('status..................................................')
    print(miner.stats)
    print('summary..................................................')
    print(miner.summary)
    print('pools..................................................')
    print(miner.pools)
    '''
    print('description..................................................')
    print(miner.description)
    print(miner.description_header)
    print(miner.description_text)
    return miner

In [ ]:
m=test()

In [ ]:
m.stats

In [ ]:
#m.reboot()

In [ ]:
m.set_credential('user','password')

In [ ]:
m.get_system_info()

In [ ]:
m.systeminfo

In [ ]:
mf = m.get_manual_freqs()

In [ ]:
type(mf)

In [ ]:
print(len(str(mf).split('\\n')))


In [ ]:
jm=json.dumps(mf.decode()).replace('"','')


In [ ]:
for l in jm.split('\\n'):
    print (l)


In [ ]:
ip__ = '192.168.1.xxx'
mm = requests.get('http://'+ ip__ +'/cgi-bin/get_system_info.cgi', auth=HTTPDigestAuth('user', 'password'))

In [ ]:
mm.content

In [ ]:
#print(m.pools['POOLS'][0])

In [ ]:
# Friendly printing 
#stats = json.dumps(miner.summary, indent=2)
#print(stats)
